**ANALISIS CHAT DE WHATSAPP**

In [0]:
import re
import io
import sys

Cargamos nuestro local Google Drive para poder acceder a los archivos que tengamos subidos ahi:


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive

Cargamos el path del log de WhatsApp que vayamos a analizar

In [0]:
path = "Jugadoras_chat.txt"

In [0]:
# Limpia los caracteres mas raros    
def replace_bad_character(line):
    return line.strip().replace(u"\u202a", "").replace(u"\u200e", "").replace(u"\u202c", "").replace(u"\xa0", " ")

Creamos una funcion que carge el archivo txt limipiando los caracteres raros:

In [0]:
def load_text(path):
    with open(path, "r", encoding="utf8") as input_file:
        text = [replace_bad_character(line) for line in input_file.readlines()]
    return text

In [0]:
def is_starting_line(line): # Comprueba si es la primera linea que explica una persona
    pattern = r"""
        (\[?)       #Zero or one open square bracket '['
        (((\d{1,2})   #1 to 2 digit date
        (/|-)       #'/' or '-' separator
        (\d{1,2})   #1 to 2 digit month
        (/|-)       #'/' or '-' separator
        (\d{2,4}))   #2 to 4 digit of year
        (,?\s)      #Zero or one comma ',' and ingle space
        ((\d{1,2})  #1 to 2 digit of hour
        (:|\.)      #Colon ':' or dot '.' separator
        (\d{2})     #2 digit of minute
        (\.|:)?     #Zero or one of dot '.' or colon ':'
        (\d{2})?    #Zero or one of 2 digits of second
        (\s[AP]M)?))  #Zero or one of ('space', 'A' or 'P', and 'M'
        (\]?\s-?\s?\s?)#Zero or one close square bracket ']', Zero or one (space and '-'), zero or one space
        (.+)        #One or more character of chat member phone number or contact name
    """
    return re.match(re.compile(pattern, re.VERBOSE), line)

In [0]:
def is_chat(body): # Comprueba que es un texto o que en realidad es un archivo, un sticker...
    pattern = r"""
            ([^:]+)#Chat member
            (:)   #Colon separator
            (.+)  #One or more charachter of message content
    """
    
    return re.match(re.compile(pattern, re.VERBOSE), body)

In [0]:
def contains_attachment(body): # Comprueba si es un archivo
    pattern_attachment = [
        ".*<Media omitted>$", #English version of android attachment
        ".*<Media tidak disertakan>$", #Indonesia version of android attachment
        ".*Pesan tidak didukung$", #Some device not recognize sticker attachment
        ".+\.vcf \(file\sterlampir\)$", #Indonesian version of android contact card,
        ".+\.vcf \(file\sattached\)$", #Indonesian version of android contact card,
        ".*image omitted$",
        ".*video omitted$",
        ".*document omitted$",
        ".*Contact card omitted$",
        ".*audio omitted$",
        ".*GIF omitted$",
        ".*sticker omitted$"
    ]
    
    for p in pattern_attachment:
        match = re.match(p, body)
        if match:
            return body
    return None

In [0]:
def is_event(body): # Evento: cambio de nombre, que alguien entra/sale del grupo...
    pattern_event = [
        "Messages to this group are now secured with end-to-end encryption\.$",
        ".+\screated this group$",
        ".+\skeluar$",
        ".+\sleft$",
        ".+\stelah bergabung menggunakan tautan undangan grup ini$",
        ".+\smenambahkan\s.+",
        ".+\sadded\s.+",
        ".+\sremoved\s.+",
        ".*You joined using this group's invite links$",
        ".+'s security code changed\.$",
        ".*telah mengganti nomor teleponnya ke nomor baru. Ketuk untuk mengirim pesan atau menambahkan nomor baru\.$",
        ".*changed their phone number to a new number. Tap to message or add the new number\.$"
    ]
    
    for p in pattern_event:
        match = re.match(p, body)
        if match:
            return match
    return None

In [0]:
def is_deleted(body): # Mensajes borrados
    p = [
        ".*This message was deleted\.$",
        ".*Pesan ini telah dihapus$"
    ]
    
    for p in p:
        match = re.match(p, body)
        if match:
            return body
    return None

In [0]:
def get_words(msg): # Cuenta las palabras mas usadas
    
    #remove non alpha content
    regex = re.sub(r"[^a-z\s]+", "", msg.lower())
    regex = re.sub(r'[^\x00-\x7f]',r'', regex)
    words = regex.split(" ")
    
    for x in words:
        if x:
            rank_word(x)
            
    return words

def rank_word(word): # Las ranquea
    
    if not word in common_words:
        popular_words[word] = popular_words.get(word, 0) + 1
        global chat_words
        chat_words += " {0}".format(word)
        
    return word

Cargamos el fichero de common words en espanol:


In [0]:
common_words = []
cw_filepath = "es_cw"
common_words = __import__(cw_filepath, globals(), locals(), [common_words]).common_words

In [0]:
chat = load_text(path)

In [0]:
chat[1:20]

['[30/7/19 22:44:18] Paola Madrid creó el grupo “Jugadoras MCFF 19/20”',
 '[30/7/19 22:44:18] Paola Madrid te añadió',
 '[30/7/19 22:49:30] Paola Madrid cambió el ícono de este grupo',
 '[30/7/19 23:13:26] Laurita 🖤⚪️: Mañana a qué hora?',
 '[30/7/19 23:15:14] Pozitaa : 10:30 empieza el entreno',
 '[30/7/19 23:15:24] Pozitaa : Bueno, hay video',
 '[31/7/19 14:35:25] Paola Madrid: Me dice Andrés que intentar ir con largos y si pueden ser negros o oscuros mejor 😊',
 '[31/7/19 21:13:12] Paola Madrid: imagen omitida',
 '[31/7/19 21:13:12] Paola Madrid: imagen omitida',
 '[31/7/19 21:13:13] Paola Madrid: imagen omitida',
 '[31/7/19 21:13:13] Paola Madrid: imagen omitida',
 '[31/7/19 21:13:14] Paola Madrid: imagen omitida',
 '[31/7/19 21:13:14] Paola Madrid: imagen omitida',
 '[31/7/19 21:15:14] Paola Madrid: 💕',
 '[31/7/19 22:24:58] Laura Fernandez Atleti: Este mensaje fue eliminado.',
 '[31/7/19 22:24:59] Laura Fernandez Atleti: Este mensaje fue eliminado.',
 '[31/7/19 22:27:34] Laura Fern

In [0]:
member_list = []
for line in chat:
    if is_starting_line(line):
        body = is_starting_line(line).group(18)
        chat = is_chat(body)
        if chat:
            member = chat.group(1)
            message_body = chat.group(3)
            if member not in member_list:
                member_list.append(member)

In [0]:
member_list

['Jugadoras MCFF 19/20',
 'Laurita 🖤⚪️',
 'Pozitaa ',
 'Paola Madrid',
 'Laura Fernandez Atleti',
 'Yaas Madrid',
 '+502 3513 4758',
 '+34 605 47 92 97',
 'Lore Reina Capi',
 '+34 663 82 70 35',
 'Maca ⚪️⚫️',
 '+34 651 76 73 32',
 'Turmo Madrid',
 'Alba Mellado ⚪️🖤',
 'Estela ⚪️🖤',
 'Carol Hernandez ⚪️🖤',
 'Itzi😇',
 '+34 673 56 85 62',
 'Clau Madrid ⚪️⚫️',
 '+34 669 08 68 04',
 'Sara Rubio Atm',
 'Julii Madrid ⚪️⚫️',
 'Mella 2.0',
 '+1 (407) 803‑9926',
 'Ale ⚪️🖤',
 'Silvia rubio',
 'Teru Bb👶🏼💜',
 'Ana Catalá ⚪️🖤',
 'Ire Cachalotis']

In [0]:
popular_words = {}

In [0]:
def parse_line(line, verbose):
    
    is_starting = is_starting_line(line)
    
    if is_starting_line:
        #Check wether the starting line is a chat or an event
        body = is_starting.group(18)
        chat = is_chat(body)
                
        if chat:
            has_attachment = contains_attachment(chat.group(3))
            increment_chat_count(chat.group(1))
            message_body = chat.group(3)
            #print(chat.groups())
            
            
            if has_attachment:
                
            else:
                if is_deleted(message_body):
                    
                else:
                   
                    #Word Count & Wordcloud
                    get_words(message_body)
                
        elif is_event(body):
        
    else:
        
        #Word Count & Wordcloud
        get_words(line)

IndentationError: ignored